In [12]:
import warnings
warnings.filterwarnings('ignore')


import pathlib
DATA_FOLDER = pathlib.Path("/home/jovyan/work/Dan/data")

# Data visualisation
import pandas as pd

# Vectorize algotrithm
from gensim.models import Word2Vec


In [13]:
PATH = DATA_FOLDER / 'pwdb/pickle/df_columns_labels.pkl'
df = pd.read_pickle(PATH)
columns = df['Concatinated Data (clean)']
columns

0      [hardship, case, fund, safety, net, selfemploy...
1      [state, support, tourism, access, financeas, t...
2      [bank, guarantees, smes, oneperson, enterprise...
3      [emergency, measures, relating, shorttime, wor...
4      [airbus, agreement, making, unworked, hours, p...
                             ...                        
930    [taxation, measures, enterprisesas, consequenc...
931    [winwinloanthe, winwin, loan, system, encourag...
932    [support, measures, businesses, closed, since,...
933    [new, loans, available, enterprises, brusselst...
934    [financial, support, teleworkers, increase, te...
Name: Concatinated Data (clean), Length: 935, dtype: object

In [14]:
# Insert our data and set minimal word count to 10, and size of each word to 300 vectors
dfVec = Word2Vec(columns, window=5, min_count=10, size=300)
w2v_dict = {w: vec for w, vec in zip(dfVec.wv.index2word, dfVec.wv.syn0)}
w2v = dfVec.wv.syn0

In [15]:
dfVec.most_similar('president')

[('established', 0.9997523427009583),
 ('cabinet', 0.9996305704116821),
 ('parliament', 0.9996006488800049),
 ('amended', 0.9995836019515991),
 ('decreelaw', 0.9995378255844116),
 ('virtue', 0.9995293617248535),
 ('introduces', 0.9995288252830505),
 ('legislation', 0.9994583129882812),
 ('declaration', 0.9994558095932007),
 ('croatia', 0.9994522929191589)]

In [18]:
dfVec.save('/home/jovyan/work/Dan/data/pwdb/word2vec/df.model')